In [1]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp


     |████████████████████████████████| 40.6MB 106kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5 libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev default-libmysqlclient-dev
The following NEW packages will be installed:
  coinor-cbc coinor-clp coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5 glpk-utils libamd2 libcolamd2
  libglpk40 libsuitesparseconfig5
0 upgraded, 12 newly installed, 0 to remove and 15 not upgraded.
Need to get 3,439 kB of archives.
After this operation, 9,832 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libamd2 amd64 1:5.1.2-2 [19.5 kB]
Get:3 

Celem zadania jest dobranie optymalnej produkcji okien drewnianych i aluminiowych oraz fabryk w których zostaną wyprodukowane w celu zmaksymalizowania zysków.

W poniższej tabeli zostały przedstawione ograniczenia co do produkcji poszczególnych okien w danej fabryce, czas ich produkcji, a także cena za okna aluminiowe i drewniane.

| Fabryka    | Okna aluminiowe | Okna drewniane | Max czas na tygodzien|
|------------|------------|------------------|---------------------|
| Fabryka 1         | 1 godzina       | Brak                | 4 godziny                |
| Fabryka 2         | Brak          | 2 godziny             | 12 godzin               |
| Fabryka 3         | 3 godziny       | 2 godziny             | 18 godzin               |


| Zysk    | Okna aluminiowe | Okna drewniane | Waluta |
|------------|------------|------------------|---------------------|
|X|3000|5000|Dolarów|

In [7]:
from pulp import *
#print(pulp.__version__)

#Dodanie modyfikacji parametrów stałych
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

#Określenie rozmiarów sliderów oraz ich opisów
style = {'description_width': '230px'}
layout= Layout(width='50%', height='auto',grid_gap="20px")

#Stworzenie sliderów wraz z ich ograniczeniami, wartościami oraz wielkościami stepów
Zysk_okna_aluminiowe_slider = FloatSlider(min=500,max=10000,value=3000,description="Zysk z okien aluminiowych",style=style,layout=layout,step=100)
Zysk_okna_drewniane_slider = FloatSlider(min=500,max=10000,value=5000,description="Zysk z okien drewnianych",style=style,layout=layout,step=100)
Fab1_godz_prod_aluminiowych_slider = FloatSlider(min=0,max=24,value=1,description="Godziny prod aluminiowych w Fabryce1",style=style,layout=layout,step=1)
Fab2_godz_prod_aluminiowych_slider = FloatSlider(min=0,max=24,value=0,description="Godziny prod aluminiowych w Fabryce2",style=style,layout=layout,step=1)
Fab3_godz_prod_aluminiowych_slider = FloatSlider(min=0,max=24,value=3,description="Godziny prod aluminiowych w Fabryce3",style=style,layout=layout,step=1)
Fab1_godz_prod_drewnianych_slider = FloatSlider(min=0,max=24,value=0,description="Godziny prod drewnianych w Fabryce1",style=style,layout=layout,step=1)
Fab2_godz_prod_drewnianych_slider = FloatSlider(min=0,max=24,value=2,description="Godziny prod drewnianych w Fabryce2",style=style,layout=layout,step=1)
Fab3_godz_prod_drewnianych_slider = FloatSlider(min=0,max=24,value=2,description="Godziny prod drewnianych w Fabryce3",style=style,layout=layout,step=1)
Fab1_max_czas_prod_slider = FloatSlider(min=0,max=50,value=4,description="Maksymalny czas prod w Fabryce1",style=style,layout=layout,step=1)
Fab2_max_czas_prod_slider = FloatSlider(min=0,max=50,value=12,description="Maksymalny czas prod w Fabryce2",style=style,layout=layout,step=1)
Fab3_max_czas_prod_slider = FloatSlider(min=0,max=50,value=18,description="Maksymalny czas prod w Fabryce3",style=style,layout=layout,step=1)

#Zdefiniowanie wartości 
def produkcja_okien(
    Zysk_okna_aluminiowe = 3000,
    Zysk_okna_drewniane = 5000,
    Fab1_godz_prod_aluminiowych = 1,
    Fab2_godz_prod_aluminiowych = 0,
    Fab3_godz_prod_aluminiowych = 3,
    Fab1_godz_prod_drewnianych = 0,
    Fab2_godz_prod_drewnianych = 2,
    Fab3_godz_prod_drewnianych = 2,
    Fab1_max_czas_prod = 4,
    Fab2_max_czas_prod = 12,
    Fab3_max_czas_prod = 18
    ):
  
    #Stworzenie problemu optymalizacji
    prob = LpProblem("Maksymalizacja_zysku",LpMaximize)

    #Zmienne decyzyjne
    x1=LpVariable("okna_aluminiowe",0,None,LpInteger)
    x2=LpVariable("okna_drewniane",0,None,LpInteger)

    #Obliczanie zysku
    prob += Zysk_okna_aluminiowe*x1 + Zysk_okna_drewniane*x2    

    #Ograniczenia
    prob += Fab1_godz_prod_aluminiowych *x1 + Fab1_godz_prod_drewnianych *x2 <= Fab1_max_czas_prod, "Maksymalny czas prod w Fabryce 1"
    prob += Fab2_godz_prod_aluminiowych *x1 + Fab2_godz_prod_drewnianych *x2 <= Fab2_max_czas_prod, "Maksymalny czas prod w Fabryce 2"
    prob += Fab3_godz_prod_aluminiowych *x1 + Fab3_godz_prod_drewnianych *x2 <= Fab3_max_czas_prod, "Maksymalny czas prod w Fabryce 3"

    if Fab1_godz_prod_aluminiowych == 0 and Fab2_godz_prod_aluminiowych == 0 and Fab3_godz_prod_aluminiowych == 0: prob += x1 == 0
    if Fab1_godz_prod_drewnianych == 0 and Fab2_godz_prod_drewnianych == 0 and Fab3_godz_prod_drewnianych == 0: prob += x2 == 0

    prob.writeLP("okna.lp")
    prob.solve()

    varsdict = {}

    for i in prob.variables():
        varsdict[i.name] = str(i.varValue)

    #!cat "okna.lp"

    #Wyświetlenie statusu oraz wyników 
    print("\n")
    print("Status:", LpStatus[prob.status])
    print("Zysk: {}$,  {}".format(value(prob.objective),varsdict))
    print()
    okna = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in prob.constraints.items()]
    print(pd.DataFrame(okna))
    print("\n")

#Przypisanie zmiennym wartości ze sliderów
interact(
    produkcja_okien,
    Zysk_okna_aluminiowe = Zysk_okna_aluminiowe_slider,
    Zysk_okna_drewniane = Zysk_okna_drewniane_slider,
    Fab1_godz_prod_aluminiowych = Fab1_godz_prod_aluminiowych_slider,
    Fab2_godz_prod_aluminiowych = Fab2_godz_prod_aluminiowych_slider,
    Fab3_godz_prod_aluminiowych = Fab3_godz_prod_aluminiowych_slider,
    Fab1_godz_prod_drewnianych = Fab1_godz_prod_drewnianych_slider,
    Fab2_godz_prod_drewnianych = Fab2_godz_prod_drewnianych_slider,
    Fab3_godz_prod_drewnianych = Fab3_godz_prod_drewnianych_slider,
    Fab1_max_czas_prod = Fab1_max_czas_prod_slider,
    Fab2_max_czas_prod = Fab2_max_czas_prod_slider,
    Fab3_max_czas_prod = Fab3_max_czas_prod_slider
)

interactive(children=(FloatSlider(value=3000.0, description='Zysk z okien aluminiowych', layout=Layout(grid_ga…

<function __main__.produkcja_okien>